<a href="https://colab.research.google.com/github/swshadle/physics/blob/master/Gaia_Clusters_HR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieving Clusters from Gaia with Hertzsprung-Russell Diagram

Learn how to retrieve [Gaia mission](https://sci.esa.int/web/gaia) data from within Google Colab. In this lab, we'll retrieve apparent brightness, location in the night sky (right ascension and declination), distance from earth, and color (how red or blue) information for one particular one star cluster, [the Pleiades](https://en.wikipedia.org/wiki/Pleiades). We'll also use the tools of astrophysics to calculate absolute magnitude, luminosity, surface temperature, and size from the Gaia-supplied data.

Partially based on [Gaia Python Tutorial: Cluster Analysis](https://gea.esac.esa.int/archive-help/tutorials/python_cluster/index.html)

If you're new to Python or Google Colab, press shift-enter on each cell to step through the code (or click the play button in the top-left of each code cell).

**STUDENTS:**
The code shows you how to retrieve and clean Gaia data, calculate new properties, and plot right ascension/declination and Hertzsprung-Russell diagrams for the Pleiades star cluster. Your assignment at the end will be to repeat this process for the Hyades cluster.

In [ ]:
# Install astroquery package to talk to Gaia database
%pip install astroquery

In [ ]:
from astroquery.gaia import Gaia

# Import data science packages
import pandas as pd

# Import NumPy to do mathy stuff
import numpy as np

print('modules imported')

In [ ]:
# Suppress warnings. Comment this out (put #'s at the start of the lines) if you wish to see the warning messages
import warnings
warnings.filterwarnings('ignore')
print('warnings suppressed')

Add timing functions to measure how long database queries take.

In [ ]:
from datetime import datetime

def timer_start():
  global start_time
  start_time = datetime.now()

def timer_stop():
  time_elapsed = datetime.now() - start_time

  da, remainder  = divmod(time_elapsed.total_seconds(), 24*3600)
  hrs, remainder = divmod(remainder, 3600)
  mins, secs = divmod(remainder, 60)

  if da:
      print(f'{int(da)} days {int(hrs)} hours {int(mins)} minutes {int(secs)} seconds elapsed')
  elif hrs:
      print(f'{int(hrs)} hours {int(mins)} minutes {int(secs)} seconds elapsed')
  elif mins:
      print(f'{int(mins)} minutes {int(secs)} seconds elapsed')
  elif secs >= 1.0:
      print(f'{int(secs)} seconds elapsed')
  else:
      print(f'{secs:.2} seconds elapsed')
        
print('timer functions loaded')

# Investigating what's available in Gaia
Load and look at the available Gaia tables.

In [ ]:
timer_start()
tables = Gaia.load_tables(only_names=False)
timer_stop()

In [ ]:
# print the ith table name and description
i=93
print(tables[i].get_qualified_name())
print(tables[i].description)

In [ ]:
# print all table names and descriptions in gaia database
for n, table in enumerate(tables):
    print(f'{n} {table.get_qualified_name()[:50]:50}', table.description.replace("\n", " "))

In [ ]:
# Build a sample query. Specifying "TOP 20" limits the results to 20 rows.
myquery = 'SELECT TOP 20 * FROM gaiadr2.gaia_source'

# Run the query and store the results
timer_start()
job = Gaia.launch_job(myquery, dump_to_file=False)
timer_stop()

Note that `myquery` uses ADQL (Astronomical Data Query Language), similar to Structured Query Language (SQL). For examples and a description of ADQL, see https://gaia.ac.uk/data/gaia-data-release-1/adql-cookbook. \
`job` now contains names and descriptions for each column in the database. (Note this is true as long as the `dump_to_file` parameter is `False` in the above call to `launch_job`).

Let's `print(job)` to see info on available columns.

In [ ]:
print(job)

In [ ]:
# Convert our AstroPy data into a pandas dataframe
sample_df = (job.get_results()).to_pandas()

In [ ]:
# Check that we got a pandas dataframe
type(sample_df)

In [ ]:
# Take a look at the first 5 rows
sample_df.head()

In [ ]:
# alternate method for looking at column names
for col in sample_df.columns:
    print(col)

# Getting data for a star cluster
Now let's retrieve data centered on the Pleiades cluster. \
<img src="https://earthsky.org/upl/2018/11/pleiades-seven-sisters-nov2019-e1572962425736.jpg" width=800 />

Image credit: Steve Paukin captured this image in his back yard in Winslow, Arizona on November 3, 2019. \
https://earthsky.org/favorite-star-patterns/pleiades-star-cluster-enjoys-worldwide-renown

Let's look up where the Pleiades are in the sky and how big the cluster appears. We'll convert to decimal degress. Here's data from https://en.wikipedia.org/wiki/Pleiades:

| Observation data (J2000 epoch) |                                   |
| ------------------------------ | --------------------------------- |
| Constellation                  | Taurus                            |
| Right ascension                | 03h 47m 24s                       |
| Declination                    | +24° 07′ 00″                      |
| Distance                       | 444 ly on average (136.2±1.2 pc)  |
| Apparent magnitude (V)         | 1.6                               |
| Apparent dimensions (V)        | 110' (arcmin)                     |
| **Physical characteristics**   |                                   |
| Other designations             | Seven Sisters, M45, Cr 42, Mel 22 |

This data is often given in degrees, minutes, and seconds. We'll make convenience functions to convert degrees, minutes, seconds and hours, minutes, seconds to decimal degrees.

In [ ]:
def dms_to_dd(d, m, s):
    dd = d + float(m)/60 + float(s)/3600
    return dd

def hms_to_dd(h, m, s):
    dd = h*15 + float(m)/4 + float(s)/240
    return dd

In the above table, right ascension is given as 03h 47m 24s. In decimal degrees, that would be:

In [ ]:
hms_to_dd(3, 47, 24)

Converting declination 24º 7' 0" to decimal degrees gives:

In [ ]:
dms_to_dd(24, 7, 0)

Finally, we will look at this part of the sky with a search radius of 110' (arcminutes). In decimal degrees, that's:

In [ ]:
dms_to_dd(0, 110, 0)

Now were ready to ask Gaia for stars at coordinates (56.85, 24.1167) with a search radius of 1.8333º. We don't need data from all columns. Let's specify:<br><br>
apparent magnitude, `phot_g_mean_mag` and rename it as `gmag` \
right ascension,    `ra` \
declination,        `dec` \
parallax,           `parallax` renamed as `plx` \
color (bp-rp),      `bp_rp` \
luminosity in solar units, `lum_val` \
effective temperature, `teff_val` \
radius in solar radii, `radius_val`

We also want absolute magnitude and distance in light years, but we can calculate these from the above data.

In [ ]:
timer_start()
job = Gaia.launch_job("SELECT phot_g_mean_mag as gmag, ra, dec, parallax as plx, bp_rp, lum_val, teff_val, radius_val \
FROM gaiadr2.gaia_source \
WHERE CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS',56.85,24.1167,1.8333))=1 \
AND parallax IS NOT NULL AND abs(parallax)>0 \
AND parallax_over_error>10 \
AND abs(pmra_error/pmra)<0.10 \
AND abs(pmdec_error/pmdec)<0.10 \
AND pmra IS NOT NULL AND abs(pmra)>0 \
AND pmdec IS NOT NULL AND abs(pmdec)>0 \
AND pmra BETWEEN 15 AND 25 \
AND pmdec BETWEEN -55 AND -40;"
, dump_to_file=False)
timer_stop()

In [ ]:
print(job)

In [ ]:
# load results into a pandas dataframe
df = (job.get_results()).to_pandas()

for col in df.columns:
    print(col)

Solving for absolute magnitude, $M$, from apparant magnitude, $m$ (which we have in column `gmag`). We'll use the formula for the distance modulus, $m - M$:
$$m - M = 5\cdot\log{d} - 5$$<br>
$$m - 5\cdot\log{d} + 5 = M$$<br>
Note that $d = 1/\omega$, where $d$ is distance measured in parsecs and $\omega$ is parallax measured in arcseconds.<br>
Our data column, `plx`, is parallax measured in milliarcseconds, so $\omega = $ `plx` $/1000$ and $d = 1000/$`plx`<br>
$$m - 5\cdot\log{\frac{1}{\omega}} + 5 = M$$<br>
$$m - 5\cdot\log{\frac{1000}{\mathtt{plx}}} + 5 = M$$<br>
$$m - 5(\log{1000} - \log{\mathtt{plx}}) + 5 = M$$<br>
$$m - 5(3 - \log{\mathtt{plx}}) + 5 = M$$<br>
$$m - 15 + 5\cdot\log{\mathtt{plx}} + 5 = M$$<br>
$$m + 5\cdot\log{\mathtt{plx}} - 10 = M$$<br>

In [ ]:
# calculate absolute magnitude and add a new column (M)
df['M'] = df['gmag'] + 5*np.log10(df['plx']) - 10

In [ ]:
df['M'].describe()

In [ ]:
# calculate distance in LY and add a new data column
# d = 1/p when distance (d) is in parsecs and parallax (p) is in arcseconds, or d = 1000/plx
# There are 3.26156 light-years (LY) in one parsec, so LY = 3261.56/plx
df['LY'] = 3261.56/df['plx']

In [ ]:
df['LY'].describe()

Now that we have absolute magnitude (column `M`), we can solve for luminosity in terms of multiples of luminosity of the sun, or $\frac{L}{L_{☉}}$, using:
$$M = +4.77 - 2.5 \cdot log\frac{L}{L_{☉}}$$<br>
$$2.5 \cdot log\frac{L}{L_{☉}} = 4.77 - M$$<br>
$$log\frac{L}{L_{☉}} = \frac{4.77 - M}{2.5}$$<br>
$$\frac{L}{L_{☉}} = 10^{\frac{4.77 - M}{2.5}}$$<br>
from http://hosting.astro.cornell.edu/academics/courses/astro201/mag_absolute.htm

In [ ]:
# add a new column ('L_sun') for luminosity in terms of multiples of solar luminosity
df['L_sun'] = np.power(10,[(4.77-m)/2.5 for m in df['M']])

In [ ]:
df['L_sun'].describe()

Our Gaia data already supplies luminosity (column `lum_val`) for some stars. Let's look at it. Note that `NaN`, for "not a number", appears when no data is provided.

In [ ]:
df['lum_val']

Let's look at just the stars that do have data (where the data is *not* null):

In [ ]:
df[df['lum_val'].isnull()==False]['lum_val']

That's a shorter list! Let's get python to compute the percentage of stars that have luminosity data.

In [ ]:
len_good = len(df[df['lum_val'].isnull()==False])
len_total = len(df)
print("That's", len_good, "stars with luminosity data out of", len_total, 
      f"({len_good/len_total:.1%})")

len_good = len(df[df['teff_val'].isnull()==False])
print("and looking ahead a bit, there are", len_good, "stars with temperature data out of", len_total, 
      f"({len_good/len_total:.1%})")

len_good = len(df[df['radius_val'].isnull()==False])
print("and", len_good, "with radius data out of", len_total, 
      f"({len_good/len_total:.1%})")

Let's make table of stars with provided luminosity (not null values) so we can compare our computed values.

In [ ]:
compare_luminosity = df.loc[df['lum_val'].isnull()==False, # the condition we want to be true
                            ['lum_val','L_sun']] # the columns that we want to put in the new table
compare_luminosity

We can plot the values to see how they compare. Seaborn library contains a convenient linear regression function, `regplot()`, that draws a scatterplot of both columns with a regression line and shaded 95% confidence interval.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

fig, axs = plt.subplots(nrows=1, figsize=(8,8))

compare_lum = df.loc[df['lum_val'].isnull()==False, ['lum_val','L_sun']]
lum = sns.regplot(compare_lum['lum_val'], compare_lum['L_sun'], ax=axs)
lum.set(xlabel='Gaia data: lum_val', ylabel='Calculated L_sun')

plt.show()

Solving for effective temperature in Kelvin, $T_K$, using:
$$T_K = \frac{5601 K}{(\mathtt{color}+0.4)^{2/3}}$$<br>
Note that this formula breaks when $\mathtt{color} = -0.4$<br>
from http://astro.physics.uiowa.edu/ITU/labs/professional-labs/photometry-of-a-globular/part-2-finding-temperature.html

In [ ]:
# check if any color values are < -0.4 or null
df[(df['bp_rp']<=-0.4) | (df['bp_rp'].isnull()==True)]

In [ ]:
# python trick: you can refer to the results of the previous code cell by the underscore character, "_".
# use the underscore "_" to grab the length of the above table.
length = len(_)
print("So we'll have to lose", length, "stars")

In [ ]:
print('# of stars before:', len(df))

In [ ]:
if length > 0:
    df = df[df['bp_rp']>-0.4]
    print(length, 'stars deleted')
else:
    print('no change needed')

In [ ]:
print('# of stars after:', len(df))

In [ ]:
# now we can add a new column ('T_K') for effective temperature in Kelvin
df['T_K'] = [5601/np.power(c+0.4,2/3) for c in df['bp_rp']]

In [ ]:
df['T_K'].describe()

Now that we have $T_K$ and $\frac{L}{L_☉}$, we can solve for radius in multiples of solar radii, $$\frac{R}{R_☉}$$<br>
using $$\frac{R}{R_☉} = \frac{\sqrt{L/L_☉}}{(T_K/T_☉)^2}$$

In [ ]:
df['R_sun']=np.around(np.sqrt(df['L_sun'])/(df['T_K']/5800)**2, decimals=2) # T_☉ = 5800

In [ ]:
df['R_sun'].describe()

In [ ]:
df['R_sun'].max()/df['R_sun'].min()

Let's see how our calculated values for effective surface temperature in Kelvin, `T_K`, and radius in multiples of solar radius, `R_sun` compare to Gaia's data (for some stars) in columns `teff_val`, and `radius_val`.

Let's look at Gaia's effective temperature and solar radius data.

In [ ]:
df['teff_val']

In [ ]:
df['radius_val']

As we did earlier with luminosity data, let's make tables of stars with provided non-null temperature and radius values and draw regression plots so we can compare our computed values.

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(8,16))

compare_temp = df.loc[df['teff_val'].isnull()==False, ['teff_val', 'T_K']]
temp = sns.regplot(compare_temp['teff_val'], compare_temp['T_K'], ax=axs[0])
temp.set(xlabel='Gaia data: teff_val', ylabel='Calculated T_K')

compare_rad = df.loc[df['radius_val'].isnull()==False, ['radius_val', 'R_sun']]
rad = sns.regplot(compare_rad['radius_val'], compare_rad['R_sun'], ax=axs[1])
rad.set(xlabel='Gaia data: radius_val', ylabel='Calculated R_sun')

plt.show()

Let's plot right ascension and declination for our cluster and display brighter stars with larger dots and more vibrant colors.

We can't use apparent brightness, `'gmag'`, directly to show brighter stars with larger dots and brighter colors because magnitude is a scale with an "inverted" sense (dimmer stars have larger values). Let's reverse the values just for this plot. 

In [ ]:
# subtract apparent brightness ('gmag') from the largest value to give the brightest stars large values and the dimmest stars small ones (the opposite of the apparent magnitude scale)
brightness = df['gmag'].max()-df['gmag']
brightness.name = 'brightness'
brightness.describe()

In [ ]:
plt.style.use('dark_background')
sns.relplot(x='ra', y='dec', height=9, aspect=1.2, alpha=0.8, edgecolor=None,
                size=brightness,
                hue=brightness,
                palette='mako', 
                data=df, 
               )
plt.title('The Pleiades in the Night Sky')

plt.show()

# HR Diagram

Let's plot color vs. absolute magnitude.

In [ ]:
sns.relplot(x='bp_rp', y='M', height=8.5, aspect=1.2, legend=None,
            #edgecolor=None,
            hue='bp_rp',
            palette='coolwarm',
            size='R_sun',
            sizes=(1,df['R_sun'].max()/df['R_sun'].min()),
            data=df
            )
plt.ylim(df['M'].max()+1, df['M'].min()-1)
plt.xlabel('Color (bp - rp)')
plt.ylabel('Absolute Magnitude')
plt.title('Hertzsprung-Russel Diagram of the Pleiades')

plt.show()

Now go look up the Hyades star cluster and
* plot ra/dec--remember to:
    * invert apparant magnitude (column `'gmag'`) for the plot
* make an HR diagram--note that you'll need to:
    * calculate absolute magnitude
    * calculate radius and
    * "clean" the data by getting rid of stars with null color data (column `'bp_rp'`)


<img src="https://www.constellation-guide.com/wp-content/uploads/2012/10/Hyades.jpg" width=800 />

Image credit: taken December 20, 2006 at 7:36pm EST by Todd Vance